In [1]:
from selenium import webdriver
from time import sleep
from scrapy.selector import Selector
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib.request
from selenium.webdriver.common.keys import Keys
from math import sin, cos, sqrt,atan2, radians
from selenium.webdriver.chrome.options import Options

pd.set_option('mode.chained_assignment', None)

chromedriver = "/home/btan/Documents/chromedriver"

chrome_options = Options()
chrome_options.add_argument("--headless")

In [2]:
driver = webdriver.Chrome(chromedriver, options = chrome_options)
# driver = webdriver.Chrome(chromedriver)

In [3]:
# initialise
def initialise_mapengine():
    # load main page
    url = 'https://www.onemap.sg/main/v2/'
    driver.get(url)
    
   # wait for the page to load
    sleep(1)
    html = driver.page_source 
    
    # click the "close" tutorial button
    button = driver.find_elements_by_xpath('//*[@alt="Got it, do not show again"]')[0]
    button.click()

In [4]:
# run page to get lat-lon
def get_loc_details(address):
    
    # click the search textbox, enter text
    button = driver.find_elements_by_xpath('//*[@id="search-text"]')[0]
    button.click()
    
    # process the address
    address = address.replace(' ST ', ' STREET ').replace(' AVE ', ' AVENUE ')
    
    # enter location
    button.send_keys(Keys.CONTROL+ "a")
    button.send_keys(address)
    sleep(0.5)
    button.send_keys(Keys.ENTER)
    sleep(0.5)
    button.send_keys(Keys.ENTER)
    
    # get the new page source
    sleep(1)
    html = driver.page_source

    while (Selector(text=html).xpath('//*[@class="latlong"]/text()').extract()[0] == []):
        sleep(0.5)
        html = driver.page_source
    
    html = driver.page_source
    
    # get the lat-lon
    buffer = Selector(text=html).xpath('//*[@class="latlong"]/text()').extract()[0]
    buffer = buffer.split(',')
    lat = buffer[0]
    lon = buffer[1].replace(' ','')
    
    nearest_station_name = Selector(text=html).xpath('//*[@class="panel-body"]/div[3]/div/text()').extract()[0].replace(' ','').replace('\n','')

    buffer = Selector(text=html).xpath('//*[@class="panel-body"]/div[3]/div/text()').extract()[1].replace('\n','')
    
    if 'km' in buffer:
        nearest_station_dist = int(float(buffer.replace(' ','').replace('km','')) * 1000)
    else:
        nearest_station_dist = int(buffer.replace(' ','').replace('m',''))
        
    bs1_name = Selector(text=html).xpath('//*[@class="col-xs-9 no-padding transport_text cursor ng-binding"]/text()').extract()[1].replace(' ','').replace('\n','')
    bs2_name = Selector(text=html).xpath('//*[@class="col-xs-9 no-padding transport_text cursor ng-binding"]/text()').extract()[2].replace(' ','').replace('\n','')
    bs3_name = Selector(text=html).xpath('//*[@class="col-xs-9 no-padding transport_text cursor ng-binding"]/text()').extract()[3].replace(' ','').replace('\n','')
    
    bs1_dist = Selector(text=html).xpath('//*[@class="col-xs-3 no-padding transport_text_right ng-binding ng-scope"]/text()').extract()[1].replace(' ','').replace('\n','')
    bs2_dist = Selector(text=html).xpath('//*[@class="col-xs-3 no-padding transport_text_right ng-binding ng-scope"]/text()').extract()[2].replace(' ','').replace('\n','')
    bs3_dist = Selector(text=html).xpath('//*[@class="col-xs-3 no-padding transport_text_right ng-binding ng-scope"]/text()').extract()[3].replace(' ','').replace('\n','')
    
    if 'km' in bs1_dist:
        bs1_dist = int(float(bs1_dist.replace('km','')) * 1000)
    else:
        bs1_dist = int(float(bs1_dist.replace('m','')))

    if 'km' in bs2_dist:
        bs2_dist = int(float(bs2_dist.replace('km','')) * 1000)
    else:
        bs2_dist = int(float(bs2_dist.replace('m','')))
    
    if 'km' in bs3_dist:
        bs3_dist = int(float(bs3_dist.replace('km','')) * 1000)
    else:
        bs3_dist = int(float(bs3_dist.replace('m','')))
    
       
    return lat, lon, nearest_station_name, nearest_station_dist, bs1_name, bs1_dist, bs2_name, bs2_dist, bs3_name, bs3_dist

In [5]:
def get_dist (lat1, lon1, lat2, lon2):
    R = 6373.0
    
    lat1 = radians(float(lat1))
    lon1 = radians(float(lon1))
    lat2 = radians(float(lat2))
    lon2 = radians(float(lon2))
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return round(distance,4)

In [6]:
def get_train_station(mode = 0, start_range = 0, end_range = 1):
    
    if mode == 0:
        print('get_train_station: specify a mode!')
        return None
    elif mode == 1:
        # load file
        try:
            train_station = pd.read_csv('./datasets/sg-train-station-updated.csv')
            return train_station
        except:
            print('get_train_station: file not found!')
            return None
    elif mode == 2:

        # read the raw file
        train_station = pd.read_csv('./datasets/sg-train-station-input.csv')
        train_station.reset_index(drop=True, inplace=True)
        # create new columns for station's lat lon
        train_station['stn_lat'] = 0.0
        train_station['stn_lon'] = 0.0
        train_station['dist_centre'] = 0.0
        # get station's lat lon, and distance to city centre (raffles place mrt)
        for station in range(len(train_station)):
            lat, lon, x1,x2,x3,x4,x5,x6,x7,x8 = get_loc_details(train_station.mrt_station_english[station] + ' MRT STATION')
            train_station.stn_lat[station] = float(lat)
            train_station.stn_lon[station] = float(lon)
            dist = get_dist(lat, lon, 1.2839, 103.8515)
            train_station.dist_centre[station] = float(dist)    
        # backup
        train_station.to_csv('./datasets/sg-train-station-output.csv')
    
        return train_station

In [7]:
def get_house_data(mode = 0, start_range = 0, end_range = -1):
    # get house data
    
    if mode == 0:
        print('get_house_data: specify a mode!')
        return None
    elif mode == 1:
        try:
            flats = pd.read_csv('./datasets/resale-flat-prices-2018-updated.csv')
            return flats
        except:
            print('get_house_data: file not found!')
            return None
    elif mode == 2:
        
        flats = pd.read_csv('./datasets/resale-flat-prices-input.csv')
        lease_empty_ind = flats[flats.remaining_lease.isnull()].index
        for row in lease_empty_ind:
            flats.loc[row, 'remaining_lease'] = int(flats.month[row][0:4]) - int(flats.lease_commence_date[row])
        flats.loc[flats[flats['remaining_lease']<0].index, 'remaining_lease'] = 0  

        flats.remaining_lease = flats.remaining_lease.astype(int)
        flats.resale_price = flats.resale_price.astype(int)
        flats.floor_area_sqm = flats.floor_area_sqm.astype(int)
        flats['transaction_year'] = flats['month'].str[0:4]
        flats['transaction_month'] = flats['month'].str[-2:]

        flats['loc_lat'] = 0.0
        flats['loc_lon'] = 0.0
        flats['nearest_mrt_name'] = ''
        flats['nearest_mrt_dist'] = ''
        flats['busstop1_name'] = ''
        flats['busstop1_dist'] = 0
        flats['busstop2_name'] = ''
        flats['busstop2_dist'] = 0
        flats['busstop3_name'] = ''
        flats['busstop3_dist'] = 0   

        flats.reset_index(drop=True, inplace=True)
        
        if end_range == -1:
            end = len(flats)
        else:
            end = end_range
            
        if end_range > len(flats):
            end = len(flats)
        
        for flat in range(start_range, end_range):
            print('Update: Processing flat {}'.format(flat))
            lat, lon, mrt_name, mrt_dist, bus1_name, bus1_dist, bus2_name, bus2_dist, bus3_name, bus3_dist = get_loc_details( flats.block[flat] + ' ' + flats.street_name[flat])
            flats.loc[flat, 'loc_lat'] = lat
            flats.loc[flat, 'loc_lon'] = lon
            flats.loc[flat, 'nearest_mrt_name'] = mrt_name
            flats.loc[flat, 'nearest_mrt_dist'] = mrt_dist
            flats.loc[flat, 'busstop1_name'] = bus1_name
            flats.loc[flat, 'busstop1_dist'] = bus1_dist
            flats.loc[flat, 'busstop2_name'] = bus2_name
            flats.loc[flat, 'busstop2_dist'] = bus2_dist
            flats.loc[flat, 'busstop3_name'] = bus3_name
            flats.loc[flat, 'busstop3_dist'] = bus3_dist

        # backup
#         flats.to_csv('./resale-flat-prices-output.csv')
        
        return flats

In [8]:
def get_school(mode = 0, start_range = 0, end_range = 1):
    
    if mode == 0:
        print('get_school: specify a mode!')
        return None
    elif mode == 1:
        # load file
        try:
            schools = pd.read_csv('./datasets/schools-cleaned-updated.csv')
            return train_station
        except:
            print('get_school: file not found!')
            return None
    elif mode == 2:

        # read the raw file
        schools = pd.read_csv('./datasets/schools-cleaned.csv')
        schools.reset_index(drop=True, inplace=True)
        # create new columns for station's lat lon
        schools['sch_lat'] = 0.0
        schools['sch_lon'] = 0.0
        schools['dist_centre'] = 0.0
        # get station's lat lon, and distance to city centre (raffles place mrt)
        for school in range(len(schools)):
            lat, lon, x1,x2,x3,x4,x5,x6,x7,x8 = get_loc_details(schools.school_name[school])
            schools.loc[school, 'sch_lat'] = lat
            schools.loc[school, 'sch_lon'] = lon
            dist = get_dist(lat, lon, 1.2839, 103.8515)
            schools.loc[school, 'dist_centre'] = dist

        # backup
        schools.reset_index(drop=True, inplace=True)
        schools.to_csv('./schools-cleaned-updated.csv')
    
        return schools

In [9]:
##########################################
# main code starts here
##########################################

In [10]:
initialise_mapengine()

In [11]:
train_station = get_train_station(2)
train_station.head()

,stn_code,mrt_station_english,mrt_line_english,stn_lat,stn_lon,dist_centre
0,NS12,Canberra,North South Line,1.443137,103.829712,17.8768


In [12]:
# schools = get_school(2)

# schools['type_primary'] = 0
# schools['type_secondary'] = 0
# schools['type_juniorcollege'] = 0
# schools['type_polytechnic'] = 0
# schools['type_university'] = 0

# schools['type_primary'] = schools.mainlevel_code.map(lambda x: 1 if 'PRIMARY' in x else 0)
# schools['type_secondary'] = schools.mainlevel_code.map(lambda x: 1 if 'SECONDARY' in x else 0)
# schools['type_juniorcollege'] = schools.mainlevel_code.map(lambda x: 1 if 'JUNIOR COLLEGE' in x else 0)
# schools['type_polytechnic'] = schools.mainlevel_code.map(lambda x: 1 if 'POLYTECHNIC' in x else 0)
# schools['type_university'] = schools.mainlevel_code.map(lambda x: 1 if 'UNIVERSITY' in x else 0)

# schools.loc[ schools[schools.mainlevel_code=='CENTRALISED INSTITUTE'].index, 'type_juniorcollege'] = 1

# schools.loc[ schools[schools.school_name=='HWA CHONG INSTITUTION'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=='DUNMAN HIGH SCHOOL'].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='DUNMAN HIGH SCHOOL'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=='RIVER VALLEY HIGH SCHOOL'].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='ANGLO-CHINESE SCHOOL (INDEPENDENT)'].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='ANGLO-CHINESE SCHOOL (INDEPENDENT)'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=='CATHOLIC HIGH SCHOOL'].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='CATHOLIC HIGH SCHOOL'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=='MARIS STELLA HIGH SCHOOL'].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='MARIS STELLA HIGH SCHOOL'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=="CHIJ ST. NICHOLAS GIRLS' SCHOOL"].index, 'type_primary'] = 1
# schools.loc[ schools[schools.school_name=="CHIJ ST. NICHOLAS GIRLS' SCHOOL"].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='NUS HIGH SCHOOL OF MATHEMATICS AND SCIENCE'].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='NUS HIGH SCHOOL OF MATHEMATICS AND SCIENCE'].index, 'type_juniorcollege']= 1
# schools.loc[ schools[schools.school_name=='NATIONAL JUNIOR COLLEGE'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=='TEMASEK JUNIOR COLLEGE'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=='RAFFLES INSTITUTION'].index, 'type_secondary'] = 1
# schools.loc[ schools[schools.school_name=='RAFFLES INSTITUTION'].index, 'type_juniorcollege'] = 1
# schools.loc[ schools[schools.school_name=="ST. JOSEPH'S INSTITUTION"].index, 'type_primary'] = 1
# schools.loc[ schools[schools.school_name=="ST. JOSEPH'S INSTITUTION"].index, 'type_secondary'] = 1

# schools.drop(index= schools[schools.type_code=='SPECIALISED SCHOOL'].index, inplace=True)
# schools.drop(index= schools[schools.type_code=='SPECIALISED INDEPENDENT SCHOOL'].index, inplace=True)

# schools.to_csv('./datasets/schools-updated.csv')

# schools.head()

In [13]:
for loop in [118]:
# for loop in range(0,1):
    if (loop % 10 == 0):
        driver.close()
        sleep(2)
#         driver = webdriver.Chrome(chromedriver)
        driver = webdriver.Chrome(chromedriver, options = chrome_options)
        initialise_mapengine()
    flats = get_house_data(2, loop*100, loop*100 + 100)
    flats = flats.loc[loop*100:loop*100+100, :]
    flats.to_csv('./datasets/output/resale-price-output-{}.csv'.format(loop))

In [14]:
# flats.head()

In [15]:
# # debugging mode

# # click the search textbox, enter text
# button = driver.find_elements_by_xpath('//*[@id="search-text"]')[0]
# button.click()

# # process the address
# # address = address.replace(' ST ', ' STREET ')

# # enter location
# button.send_keys(Keys.CONTROL+ "a")
# button.send_keys('681C jurong west')
# button.send_keys(Keys.ENTER)

# sleep(1)

# html = driver.page_source

In [16]:
# lat1, lon1 = get_latlon('251 bishan street 22')
# lat2, lon2 = get_latlon('681c jurong west')

# dist = get_dist(lat1, lon1, lat2, lon2)
# print(dist)

In [17]:
driver.close()